### ***************Question and Answer tool in Retail domain***************

#### End-to-End LLM project using the technologies 
1. LangChain
2. Hugging Face
3. Streamlit
4. Chroma
5. MySQL
6. Google PaLm

##### Description:
Happy Tees is a clothing store where their data is stored in MySQL database.

Output:
I built a tool similar to ChatGPT where I can ask a question in natural human language. It'll somehow convert the question into a SQL query and execute it on the database.

##### Project requirements: 
1. Happy_Tees database in MySQL
   a. t_shirts(Database): maintains the inventory count that includes price per each unit.
   b. discounts(Database): Discount for each piece in t_shirts database.

##### Technical Architecture:
1. Google PaLm(LLM) available in LangChain framework: Responsible for converting natural language question to a SQL Query.
2. SQLDatabaseChain class in LangChain framework.
3. Few short learning: To handle complex questions
       Preparing a training dataset where we have same question and corresponding SQL query.
4. Hugging Face library: To convert the training dataset into embedding vectors.
5. ChromaDB(open source vector database): To store the embedded vectors.


In [48]:
!pip install -U langchain-community

In [49]:
!pip install langchain

In [50]:
!pip install google-generativeai

In [51]:
!pip install pymysql

In [52]:
!pip install mysql-connector-python

In [53]:
!pip install cryptography

In [54]:
!pip install PyMySQL[rsa]

In [55]:
!pip install langchain_experimental

In [56]:
!pip install sentence_transformers

In [57]:
!pip install chromadb

# **************Implementation**************

In [13]:
# Create an API Key for Google PaLm
'''Souce: https://aistudio.google.com/app/apikey'''

from langchain.llms import GooglePalm

In [14]:
api_key = "secret"

In [15]:
llm = GooglePalm(google_api_key = api_key, temperature = 0.2)

C:\Users\tejas\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
india_description = llm("Give me a brief description about India")
print(india_description)

C:\Users\tejas\AppData\Local\Programs\Python\Python312\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


India is a country in South Asia. It is the seventh-largest country by area and the second-most populous country with over 1.3 billion people. India is a diverse country with a variety of languages, religions, and cultures. The capital of India is New Delhi.


In [17]:
from langchain.utilities import SQLDatabase

In [18]:
db_user = "root"
db_password = "MySQL_Workbench13"
db_host = "localhost"
db_name = "Happy_Tees"
db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}", sample_rows_in_table_info = 3)

In [19]:
print(db.table_info)


CREATE TABLE discounts (
	discount_id INTEGER NOT NULL AUTO_INCREMENT, 
	t_shirt_id INTEGER NOT NULL, 
	pct_discount DECIMAL(5, 2), 
	PRIMARY KEY (discount_id), 
	CONSTRAINT discounts_ibfk_1 FOREIGN KEY(t_shirt_id) REFERENCES t_shirts (t_shirt_id), 
	CONSTRAINT discounts_chk_1 CHECK ((`pct_discount` between 0 and 100))
)COLLATE utf8mb4_0900_ai_ci ENGINE=InnoDB DEFAULT CHARSET=utf8mb4

/*
3 rows from discounts table:
discount_id	t_shirt_id	pct_discount
1	1	10.00
2	2	15.00
3	3	20.00
*/


CREATE TABLE t_shirts (
	t_shirt_id INTEGER NOT NULL AUTO_INCREMENT, 
	brand ENUM('Van Huesen','Levi','Nike','Adidas') NOT NULL, 
	color ENUM('Red','Blue','Black','White') NOT NULL, 
	size ENUM('XS','S','M','L','XL') NOT NULL, 
	price INTEGER, 
	stock_quantity INTEGER NOT NULL, 
	PRIMARY KEY (t_shirt_id), 
	CONSTRAINT t_shirts_chk_1 CHECK ((`price` between 10 and 50))
)COLLATE utf8mb4_0900_ai_ci ENGINE=InnoDB DEFAULT CHARSET=utf8mb4

/*
3 rows from t_shirts table:
t_shirt_id	brand	color	size	price	stock

In [20]:
'''SQLDatabaseChain - execute SQL queries against a database using natural language queries'''

from langchain_experimental.sql import SQLDatabaseChain

In [21]:
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose = True)
question_one = db_chain("How many T-Shirts are we left with for a brand of Nike, with a size of extra small, with a color of white ?")

C:\Users\tejas\AppData\Local\Programs\Python\Python312\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new SQLDatabaseChain chain...
How many T-Shirts are we left with for a brand of Nike, with a size of extra small, with a color of white ?
SQLQuery:SELECT stock_quantity FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'
SQLResult: [(50,)]
Answer:50
> Finished chain.


In [22]:
question_one

{'query': 'How many T-Shirts are we left with for a brand of Nike, with a size of extra small, with a color of white ?',
 'result': '50'}

In [23]:
question_two = db_chain("How much is the price of all small sized t-shirts in the inventory ?")



> Entering new SQLDatabaseChain chain...
How much is the price of all small sized t-shirts in the inventory ?
SQLQuery:SELECT SUM(price) FROM t_shirts WHERE size = 'S'
SQLResult: [(Decimal('205'),)]
Answer:205
> Finished chain.


In [24]:
question_two = db_chain("SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'")



> Entering new SQLDatabaseChain chain...
SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'
SQLQuery:SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'
SQLResult: [(Decimal('12250'),)]
Answer:12250
> Finished chain.


In [25]:
question_two

{'query': "SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'",
 'result': '12250'}

In [26]:
question_three = db_chain("How many white colour Levi's T-Shirts are available currently ?")



> Entering new SQLDatabaseChain chain...
How many white colour Levi's T-Shirts are available currently ?
SQLQuery:SELECT stock_quantity FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLResult: [(92,), (35,), (52,), (39,)]
Answer:92
> Finished chain.


In [27]:
question_three = db_chain("SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'")




> Entering new SQLDatabaseChain chain...
SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLQuery:SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLResult: [(Decimal('218'),)]
Answer:218
> Finished chain.


In [28]:
question_three

{'query': "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'",
 'result': '218'}

#### Few Shot Learning

In [29]:
'''
Step-1: Provide Question and Query pairs.
Step-2: Convert it into embeddings (Using HuggingFace)
'''
#Step=1
wrong_shots = [
        {
        'Question':"How much is the price of all small sized t-shirts in the inventory ?",
        'SQLQuery':"SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'",
        'SQLResult':"Result of the SQL Query",
        'Answer': '12263'},
    
    {
        'Question':"How many white colour Levi's T-Shirts are available currently ?",
        'SQLQuery':"SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'",
        'SQLResult':"Result of the SQL Query",
        'Answer':'221'}
                ]
type(wrong_shots)

list

In [30]:
#Step-2
from langchain.embeddings import HuggingFaceBgeEmbeddings

In [31]:
embeddings = HuggingFaceBgeEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

#### ********Vector database********

In [32]:
text =[" ".join(example.values()) for example in wrong_shots]
text

["How much is the price of all small sized t-shirts in the inventory ? SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S' Result of the SQL Query 12263",
 "How many white colour Levi's T-Shirts are available currently ? SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White' Result of the SQL Query 221"]

In [33]:
text[0]

"How much is the price of all small sized t-shirts in the inventory ? SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S' Result of the SQL Query 12263"

In [34]:
from langchain.vectorstores import Chroma

In [35]:
vector = Chroma.from_texts(text, embedding= embeddings, metadatas= wrong_shots)
vector

In [36]:
'''Let's try'''
from langchain.prompts import SemanticSimilarityExampleSelector

In [37]:
example = SemanticSimilarityExampleSelector(vectorstore= vector, k = 1)
example.select_examples({'Question' : "How many Levi's T-Shirts are available currently"})

[{'Answer': '221',
  'Question': "How many white colour Levi's T-Shirts are available currently ?",
  'SQLQuery': "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'",
  'SQLResult': 'Result of the SQL Query'}]

In [38]:
'''Instruct the LLM to follow the rules based on scenario'''
from langchain.chains.sql_database.prompt import PROMPT_SUFFIX, _mysql_prompt

In [39]:
rules_to_follow_LLM = """You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
    Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
    Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
    Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
    Pay attention to use CURDATE() function to get the current date, if the question involves "today".
    
    Use the following format:
    
    Question: Question here
    SQLQuery: Query to run with no pre-amble
    SQLResult: Result of the SQLQuery
    Answer: Final answer here
    
    No pre-amble.
    """
rules_to_follow_LLM

'You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.\n    Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.\n    Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.\n    Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.\n    Pay attention to use CURDATE() function to get the current date, if the question involves "today".\n    \n    Use the following format:\n    \n    Question: Question here\n    S

In [40]:
print(PROMPT_SUFFIX)

Only use the following tables:
{table_info}

Question: {input}


In [41]:
from langchain.prompts.prompt import PromptTemplate
example_prompt = PromptTemplate(
        input_variables=["Question", "SQLQuery", "SQLResult","Answer"],
        template="\nQuestion: {Question}\nSQLQuery: {SQLQuery}\nSQLResult: {SQLResult}\nAnswer: {Answer}"
    )

In [42]:
from langchain.prompts import FewShotPromptTemplate

In [43]:
few_shot_prompt = FewShotPromptTemplate(
        example_selector=example,
        example_prompt=example_prompt,
        prefix=_mysql_prompt,
        suffix=PROMPT_SUFFIX,
        input_variables=["input", "table_info", "top_k"], #These variables are used in the prefix and suffix
    )

In [44]:
new_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True, prompt=few_shot_prompt)

####   **********Now, Test with the failing questions**********

In [45]:
new_chain("How much is the price of all small sized t-shirts in the inventory ?")



> Entering new SQLDatabaseChain chain...
How much is the price of all small sized t-shirts in the inventory ?
SQLQuery:SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'
SQLResult: [(Decimal('12250'),)]
Answer:12250
> Finished chain.


{'query': 'How much is the price of all small sized t-shirts in the inventory ?',
 'result': '12250'}

In [46]:
new_chain("How many white colour Levi's T-Shirts are available currently ?")



> Entering new SQLDatabaseChain chain...
How many white colour Levi's T-Shirts are available currently ?
SQLQuery:SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLResult: [(Decimal('218'),)]
Answer:218
> Finished chain.


{'query': "How many white colour Levi's T-Shirts are available currently ?",
 'result': '218'}